<p style="font-family:verdana;font-size:200%;text-align:center;">Open API : APT Price Seoul</p>

- 지역코드와 거래년월을 입력하면 해당 거래 데이터를 수집하는 사용자 정의 함수를 생성합니다.
- 지역코드와 거래년월을 변경하면 원하는 데이터를 빠르게 수집할 수 있습니다.

## 작업경로 설정

In [1]:
# 관련 라이브러리를 호출합니다.
import os

In [2]:
# 현재 작업경로를 확인합니다.
os.getcwd()

'C:\\Users\\kim0m\\side_project\\PythonRPA-main\\code'

In [3]:
# data 폴더로 작업경로를 변경합니다.
os.chdir(path = '../data')

In [4]:
# 작업경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

['APT_List_Seoul_2020.xlsx',
 'APT_Price_Seoul_2020.xlsx',
 'Area_Code.xlsx',
 'covid_20210415.xlsx',
 'daum_news_20210415.xlsx',
 'Market_Index_20210415.xlsx',
 'Market_Index_Form.xlsx',
 'Naver_APT_Detail.xlsx',
 'Open_API_Key.txt',
 'test.txt']

## 아파트 거래 데이터 수집하는 사용자 정의 함수 생성

In [5]:
# 관련 라이브러리를 호출합니다.
import requests
import xmltodict
import pandas as pd

# 지역코드와 거래년월을 입력하면 데이터를 수집하는 사용자 정의 함수를 생성합니다.
def aptPrice(areaCd, ymonth, apiKey):
    
    # 요청 URL의 성분을 문자열로 각각 생성합니다.
    host = 'http://openapi.molit.go.kr:8081'
    path = '/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc'
    oper = '/getRTMSDataSvcAptTrade'
    
    # Query String을 생성합니다.
    query = {'LAWD_CD': areaCd, 'DEAL_YMD': ymonth, 'serviceKey': apiKey}
    
    # HTTP 요청을 실행합니다.
    res = requests.get(url = host + path + oper, params = query)
    
    # str 자료형을 collections.OrderedDict 자료형으로 변환합니다.
    dat = xmltodict.parse(xml_input = res.text)
    
    # data를 데이터프레임으로 변환합니다.
    df = pd.DataFrame(data = dat['response']['body']['items']['item'])
    
    # 데이터프레임을 반환합니다.
    return df

In [6]:
# 공공데이터포털 API Key가 포함된 텍스트 파일을 읽습니다.
apiKey = open(file = 'Open_API_Key.txt', mode = 'r').read()

In [7]:
# 예제 데이터로 실습
apt = aptPrice(areaCd = '11110', ymonth = '201512', apiKey = apiKey)

In [8]:
# 처음 다섯 행만 출력합니다.
apt.head()

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"82,500",2008,2015,사직동,광화문풍림스페이스본(101동~105동),12,10,94.51,9,11110,11,None,None
1,"60,000",1981,2015,당주동,롯데미도파광화문빌딩,12,22,149.95,145,11110,8,None,None
2,"130,000",2004,2015,내수동,킹스매너,12,8,194.43,110-15,11110,6,None,None
3,"105,000",2004,2015,내수동,경희궁의아침2단지,12,14,124.17,71,11110,8,None,None
4,"120,000",2003,2015,내수동,경희궁 파크팰리스,12,24,146.33,95,11110,4,None,None


## 지역코드 준비

In [9]:
# 서울특별시 자치구 정보가 포함된 엑셀파일을 읽습니다.
dong = pd.read_excel(io = 'Area_Code.xlsx')

In [10]:
# 처음 10행만 출력합니다.
dong.head(n = 10)

,code,sido,sigg
0,1111000000,서울특별시,종로구
1,1114000000,서울특별시,중구
2,1117000000,서울특별시,용산구
3,1120000000,서울특별시,성동구
4,1121500000,서울특별시,광진구
5,1123000000,서울특별시,동대문구
6,1126000000,서울특별시,중랑구
7,1129000000,서울특별시,성북구
8,1130500000,서울특별시,강북구
9,1132000000,서울특별시,도봉구


In [11]:
# 데이터프레임의 정보를 확인합니다.
dong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    25 non-null     int64 
 1   sido    25 non-null     object
 2   sigg    25 non-null     object
dtypes: int64(1), object(2)
memory usage: 728.0+ bytes


## 반복문 실행

In [12]:
# 거래년월을 설정합니다.
ymonth = '202001'

In [13]:
# 관련 라이브러리를 호출합니다.
# [참고] 웹 크롤링을 실행할 때 잠시 멈춤 동작을 추가해야 합니다.
from time import *

# 최종 결과를 저장할 빈 데이터프레임을 생성합니다.
apt = pd.DataFrame()

# 반복문을 실행합니다.
for i in range(len(dong)):
    
    # 지역명을 설정합니다.
    areaNm = dong['sido'][i] + ' ' + dong['sigg'][i]
    
    # 현재 진행상황을 출력합니다. 이 코드를 생략하면 진행상황을 알 수 없게 됩니다.
    print(f'현재 {areaNm} 데이터 수집 중!')
    
    # 지역코드를 설정합니다. 정수를 문자열로 변경한 다음, 처음 5글자만 잘라냅니다.
    areaCd = dong['code'][i].astype(str)[0:5]
    
    # 거래 건수가 없으면 aptPrice() 함수는 에러를 발생하고 반복문이 중단됩니다.
    # 따라서 try, except 문으로 에러가 발생하면 다음 자치구로 이동시킵니다.
    try:
        
        # 서울특별시 자치구 단위로 거래 데이터를 수집합니다.
        df = aptPrice(areaCd = areaCd, ymonth = ymonth, apiKey = apiKey)
        
        # 필요한 컬럼을 추가합니다.
        df['시도'] = dong['sido'][i]
        df['시군구'] = dong['sigg'][i]
        
        # 최종 결과 데이터프레임에 apt를 추가합니다.
        # [참고] 인덱스가 중복되면 에러가 발생합니다.
        apt = pd.concat(objs = [apt, df], ignore_index = True)
    
    except:
        print('>> 거래 건수가 없습니다. 다음 자치구로 이동합니다!')
        next
    
    # 1초 간 멈춥니다.
    sleep(1)

# 데이터 수집 완료
print('모든 데이터를 수집했습니다!')

현재 서울특별시 종로구 데이터 수집 중!
현재 서울특별시 중구 데이터 수집 중!
현재 서울특별시 용산구 데이터 수집 중!
현재 서울특별시 성동구 데이터 수집 중!
현재 서울특별시 광진구 데이터 수집 중!
현재 서울특별시 동대문구 데이터 수집 중!
현재 서울특별시 중랑구 데이터 수집 중!
현재 서울특별시 성북구 데이터 수집 중!
현재 서울특별시 강북구 데이터 수집 중!
현재 서울특별시 도봉구 데이터 수집 중!
현재 서울특별시 노원구 데이터 수집 중!
현재 서울특별시 은평구 데이터 수집 중!
현재 서울특별시 서대문구 데이터 수집 중!
현재 서울특별시 마포구 데이터 수집 중!
현재 서울특별시 양천구 데이터 수집 중!
현재 서울특별시 강서구 데이터 수집 중!
현재 서울특별시 구로구 데이터 수집 중!
현재 서울특별시 금천구 데이터 수집 중!
현재 서울특별시 영등포구 데이터 수집 중!
현재 서울특별시 동작구 데이터 수집 중!
현재 서울특별시 관악구 데이터 수집 중!
현재 서울특별시 서초구 데이터 수집 중!
현재 서울특별시 강남구 데이터 수집 중!
현재 서울특별시 송파구 데이터 수집 중!
현재 서울특별시 강동구 데이터 수집 중!
모든 데이터를 수집했습니다!


In [14]:
# 최종 결과 데이터프레임의 정보를 확인합니다.
apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6506 entries, 0 to 6505
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   거래금액     6506 non-null   object
 1   건축년도     6506 non-null   object
 2   년        6506 non-null   object
 3   법정동      6506 non-null   object
 4   아파트      6506 non-null   object
 5   월        6506 non-null   object
 6   일        6506 non-null   object
 7   전용면적     6506 non-null   object
 8   지번       6504 non-null   object
 9   지역코드     6506 non-null   object
 10  층        6506 non-null   object
 11  해제사유발생일  0 non-null      object
 12  해제여부     0 non-null      object
 13  시도       6506 non-null   object
 14  시군구      6506 non-null   object
dtypes: object(15)
memory usage: 762.5+ KB


In [16]:
apt.groupby('시군구').count()['거래금액'].sort_values(ascending =False)

시군구
노원구     815
성북구     449
구로구     443
강서구     425
도봉구     367
강동구     336
은평구     313
강북구     313
영등포구    307
동대문구    287
양천구     241
중랑구     237
관악구     235
서대문구    227
송파구     221
동작구     208
성동구     206
마포구     182
강남구     131
금천구     125
서초구     123
용산구     103
중구       84
종로구      65
광진구      63
Name: 거래금액, dtype: int64

## 엑셀 파일로 저장

In [17]:
# 저장할 엑셀 파일명을 생성합니다.
file = f'APT_Price_Seoul_{ymonth}.xlsx'
file

'APT_Price_Seoul_202001.xlsx'

In [18]:
# 엑셀 파일로 저장합니다.
apt.to_excel(excel_writer = file, index = False)

In [19]:
# 작업경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

['APT_List_Seoul_2020.xlsx',
 'APT_Price_Seoul_2020.xlsx',
 'APT_Price_Seoul_202001.xlsx',
 'Area_Code.xlsx',
 'covid_20210415.xlsx',
 'daum_news_20210415.xlsx',
 'Market_Index_20210415.xlsx',
 'Market_Index_Form.xlsx',
 'Naver_APT_Detail.xlsx',
 'Open_API_Key.txt',
 'test.txt']

<p style="font-family:verdana;font-size:200%;text-align:center;">End of Document</p>